In [3]:
import nfl_data_py as nfl
import pandas as pd
import json

In [4]:
years = [2023]
# ['ALL','REG','POST']
s_type = 'REG'

In [5]:
def get_data(years,s_type):
    weekly_data = nfl.import_weekly_data(years)
    season_data = nfl.import_seasonal_data(years,s_type)

    data_merge = pd.merge(season_data, weekly_data[['player_id','season','season_type','player_display_name','recent_team','position_group','headshot_url']].drop_duplicates(subset=['player_id','season','season_type']), on=['player_id','season','season_type'], how='left')
    filtered_positions = data_merge[data_merge['position_group'].isin(['QB', 'WR','RB','TE'])]

    return filtered_positions

In [6]:
def filter_data(data,tom):
    # Get data from json
    with open('dictionaries/types_of_metrics.json', 'r') as archivo:
        types_of_metrics = json.load(archivo)

    # Filter data depending of the type of metrics the user needs
    filtered_data = data[types_of_metrics[tom] + types_of_metrics['Player Info']]
    return filtered_data

In [7]:
data = get_data(years,s_type)

Downcasting floats.


In [8]:
df2 = filter_data(data,'Passing Metrics')

In [8]:
df2

,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,player_display_name,recent_team,position_group,games,headshot_url
0,0,1,0.0,0,0.0,1.0,0,0,17.0,0.0,0.0,-2.031960,0,Aaron Rodgers,NYJ,QB,1,https://static.www.nfl.com/image/private/f_aut...
1,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.000000,0,Marcedes Lewis,CHI,TE,5,https://static.www.nfl.com/image/private/f_aut...
2,123,204,1616.0,13,8.0,8.0,3,1,1910.0,678.0,60.0,-1.607587,1,Joe Flacco,CLE,QB,5,https://static.www.nfl.com/image/upload/f_auto...
3,326,521,3965.0,24,11.0,30.0,0,0,4033.0,1890.0,187.0,55.032108,1,Matthew Stafford,LA,QB,15,https://static.www.nfl.com/image/private/f_aut...
4,23,42,231.0,0,2.0,1.0,0,0,344.0,80.0,14.0,-8.780773,0,Brian Hoyer,LV,QB,2,https://static.www.nfl.com/image/private/f_aut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,315,527,2877.0,11,10.0,62.0,9,6,4009.0,1300.0,133.0,-160.326442,1,Bryce Young,CAR,QB,16,https://static.www.nfl.com/image/private/f_aut...
584,149,255,1808.0,8,4.0,28.0,5,2,2688.0,746.0,81.0,-13.886491,1,Will Levis,TEN,QB,9,https://static.www.nfl.com/image/private/f_aut...
585,319,499,4108.0,23,5.0,38.0,6,3,4481.0,1762.0,188.0,64.826173,0,C.J. Stroud,HOU,QB,15,https://static.www.nfl.com/image/private/f_aut...
586,50,84,577.0,3,1.0,7.0,1,0,676.0,297.0,31.0,4.587791,2,Anthony Richardson,IND,QB,4,https://static.www.nfl.com/image/private/f_aut...


In [12]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

def passing_metrics_clustering(data,years):
    # Preprocesamiento de datos
    features = data.drop(columns=['player_display_name', 'recent_team', 'position_group', 'headshot_url'])
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(features)

    # Aplicar K-means
    kmeans = KMeans(n_clusters=2, random_state=42)  # Puedes ajustar el número de clusters
    data['cluster'] = kmeans.fit_predict(scaled_features)
    data.to_csv('passing_metrics_2023.csv')

    plt.figure()
    sns.pairplot(df2,hue='cluster')
    plt.title('Passing Metrics Pairplot by Clusters')
    plt.savefig('passing_metrics_pairplot_'+str(years)+'.png')

    return data